robustness check with reiprocity from 2010 and 2015

In [95]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

In [214]:
# define path: insert the path where the SOEP data is stored on your computer here
from pathlib import Path
# Maxie: /Volumes/dohmen_soep/SOEP-CORE.v36eu_STATA/Stata/raw
# Max: C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/SOEP-Data/Stata/raw
data_folder = Path("/Volumes/dohmen_soep/SOEP-CORE.v36eu_STATA/Stata/raw")
# define relevant subsets of SOEP-data
file_names = ['vp', 'bap', 'bfp']

file_paths = [data_folder / f"{file_name}.dta" for file_name in file_names]
# some controls are in gen data
file_paths_2 = [data_folder / f"{file_name}gen.dta" for file_name in file_names]

In [215]:
data05 = pd.read_stata(file_paths[0], columns=["pid","hid", "syear","vp12602", "vp12603", "vp12605"]).set_index(['pid', 'hid'])
df_05 = data05.rename(columns={ 'vp12602': 'take_revenge', 'vp12603': 'similar_problems', 'vp12605': 'insult_back'})


In [216]:
data10= pd.read_stata(file_paths[1], columns=["pid", "hid", "syear",'bap12402','bap12403','bap12405']).set_index(['pid', 'hid'])
df_10 = data10.rename(columns = {'bap12402': 'take_revenge','bap12403': 'similar_problems','bap12405': 'insult_back'})

In [217]:
data15= pd.read_stata(file_paths[2], columns=["pid", "hid", "syear",'bfp0602','bfp0603','bfp0605']).set_index(['pid', 'hid'])
df_15 = data15.rename(columns = {'bfp0602': 'take_revenge', 'bfp0603': 'similar_problems', 'bfp0605': 'insult_back'})


In [218]:
reciprocity_questions_mapping_05 = {
    '[1] Trifft ueberhaupt nicht zu': 1,
    '[2] Skala 1-7': 2,
    '[3] Skala 1-7': 3,
    '[4] Skala 1-7': 4,
    '[5] Skala 1-7': 5,
    '[6] Skala 1-7': 6,
    '[7] Trifft voll zu': 7,
    '[-1] keine Angabe': np.nan,

}

df_05[["similar_problems","take_revenge","insult_back"]] = df_05[["similar_problems","take_revenge","insult_back"]].apply(lambda x: x.map(reciprocity_questions_mapping_05))

reciprocity_questions_mapping_10 = {
    '[1] Trifft ueberhaupt nicht zu': 1,
    '[2] 2 auf Skala 1-7': 2,
    '[3] 3 auf Skala 1-7': 3,
    '[4] 4 auf Skala 1-7': 4,
    '[5] 5 auf Skala 1-7': 5,
    '[6] 6 auf Skala 1-7': 6,
    '[7] Trifft voll zu': 7,
    '[-1] keine Angabe': np.nan,
    '[-5] In Fragebogenversion nicht enthalten': np.nan,

}

df_10[["similar_problems","take_revenge","insult_back"]] = df_10[["similar_problems","take_revenge","insult_back"]].apply(lambda x: x.map(reciprocity_questions_mapping_10))

reciprocity_questions_mapping_15 = {
    '[1] 1 Trifft ueberhaupt nicht zu, (Skala 1-7)': 1,
    '[2] 2 auf Skala 1-7': 2,
    '[3] 3 auf Skala 1-7': 3,
    '[4] 4 auf Skala 1-7': 4,
    '[5] 5 auf Skala 1-7': 5,
    '[6] 6 auf Skala 1-7': 6,
    '[7] 7 Trifft voll zu, (Skala 1-7)': 7,
    '[-1] keine Angabe': -1,

}

df_15[["similar_problems","take_revenge","insult_back"]] = df_15[["similar_problems","take_revenge","insult_back"]].apply(lambda x: x.map(reciprocity_questions_mapping_15))


In [219]:
df = df_10.merge(df_05,how='left', left_on=['pid', 'hid'], right_on=['pid', 'hid'])
df_reciprocity = df_15.merge(df, how='left', left_on=['pid', 'hid'], right_on=['pid', 'hid'])
#categoricals need to be transoformed to integers, otherwise dopping negatives is not possible


In [220]:
df_reciprocity['avg_rec'] = df_reciprocity[['take_revenge', 'similar_problems', 'insult_back', 'take_revenge_x', 'similar_problems_x', 'insult_back_x', 'take_revenge_y', 'similar_problems_y', 'insult_back_y']].mean(axis=1, skipna = True)

In [222]:
df_avg_rec = df_reciprocity.loc[:,['avg_rec']]
df_avg_rec

,,avg_rec
pid,hid,
901,94,3.444444
1501,159,1.888889
1601,167,2.888889
2301,230,2.666667
2302,65889,4.111111
...,...,...
35030302,3503037,2.000000
35030502,3503053,4.333333
35031602,3503169,3.000000


In [223]:
df_avg_rec.to_csv('/Users/maxieschulze/Documents/Dokumente - MacBook Pro von Maxie/5. Semester/Research Module/ResearchModule/src/data_management/rec_avgyears.csv')